In [22]:
import ccxt
import pandas as pd
import json

with open('keys.json', 'r') as f:
    keys = json.load(f)
api_key = keys['api_key']
api_secret = keys['api_secret']

exchange = ccxt.binance({
    'apiKey': api_key,
    'secret': api_secret,
    'enableRateLimit': True
})
exchange.load_markets()

AuthenticationError: binance {"code":-2008,"msg":"Invalid Api-Key ID."}

In [17]:
exchange.set_sandbox_mode(True)

In [3]:
import ccxt
import pandas as pd

def retrieve_candles(symbol, timeframe, limit):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)

    df_min = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df_min['timestamp'] = pd.to_datetime(df_min['timestamp'], unit='ms')
    return df_min

In [7]:
live_prices = retrieve_candles('BNB/USDT', '1h', 200)
live_prices

,timestamp,open,high,low,close,volume
0,2025-07-02 11:00:00,656.90,657.13,656.90,657.10,25.168
1,2025-07-02 12:00:00,657.11,821.80,655.39,656.10,4891.273
2,2025-07-02 13:00:00,656.13,658.27,656.13,657.76,4527.769
3,2025-07-02 14:00:00,657.81,659.44,657.40,659.27,5927.051
4,2025-07-02 15:00:00,659.29,661.99,658.81,661.66,4849.988
...,...,...,...,...,...,...
165,2025-07-09 08:00:00,662.61,662.87,660.95,660.95,2089.409
166,2025-07-09 09:00:00,660.90,660.90,659.76,660.18,2165.653
167,2025-07-09 10:00:00,660.18,660.63,659.16,660.58,2854.144
168,2025-07-09 11:00:00,660.59,662.15,660.59,661.30,2759.139


In [8]:
import pandas_ta as ta

live_prices['RSI'] = ta.rsi(live_prices['close'], length=14)

live_prices['MA20'] = live_prices['close'].rolling(window=20).mean()
live_prices['MA50'] = live_prices['close'].rolling(window=50).mean()

live_prices = live_prices.dropna()
live_prices

,timestamp,open,high,low,close,volume,RSI,MA20,MA50
49,2025-07-04 12:00:00,659.63,660.09,658.31,658.73,2508.116,45.546295,660.4065,660.7520
50,2025-07-04 13:00:00,658.74,659.69,655.73,655.73,4903.596,37.766770,660.2360,660.7246
51,2025-07-04 14:00:00,655.67,655.91,653.31,655.52,5297.226,37.286665,659.9875,660.7130
52,2025-07-04 15:00:00,655.51,655.56,652.22,653.07,5816.454,32.151451,659.5670,660.6192
53,2025-07-04 16:00:00,652.79,652.89,650.22,652.05,6812.605,30.281618,659.0760,660.4748
...,...,...,...,...,...,...,...,...,...
165,2025-07-09 08:00:00,662.61,662.87,660.95,660.95,2089.409,49.292247,660.9405,660.6748
166,2025-07-09 09:00:00,660.90,660.90,659.76,660.18,2165.653,46.371099,660.8690,660.6228
167,2025-07-09 10:00:00,660.18,660.63,659.16,660.58,2854.144,48.092028,660.9790,660.5694
168,2025-07-09 11:00:00,660.59,662.15,660.59,661.30,2759.139,51.131843,661.1360,660.5436


In [9]:
import plotly.graph_objects as go
import plotly.subplots as sp
import pandas as pd
from datetime import datetime

fig = sp.make_subplots(rows=2, cols=1, shared_xaxes=True, 
                       row_heights=[0.7, 0.3], vertical_spacing=0.22,
                       subplot_titles=("Candlestick Chart with 20-period MA", "RSI"))

fig.add_trace(go.Candlestick(x=live_prices['timestamp'],
                             open=live_prices['open'],
                             high=live_prices['high'],
                             low=live_prices['low'],
                             close=live_prices['close'],
                             name='Candlestick'), row=1, col=1)

fig.add_trace(go.Scatter(x=live_prices['timestamp'], y=live_prices['MA20'],
                         mode='lines', line=dict(color='blue', width=2),
                         name='20-period MA'), row=1, col=1)

fig.add_trace(go.Scatter(x=live_prices['timestamp'], y=live_prices['RSI'],
                         mode='lines', line=dict(color='orange', width=2),
                         name='RSI'), row=2, col=1)

fig.update_layout(
    width=900,
    height=700)
fig.show()

In [10]:
if 'order_book' in globals():
    del order_book

column_names = ['Date', 'Time', 'Symbol', 'Side', 'Price', 'Quantity', 'Value', 'executedQty', 'Commission']

order_book = pd.DataFrame(columns=column_names)
order_book

,Date,Time,Symbol,Side,Price,Quantity,Value,executedQty,Commission


In [11]:
def append_order_book(order):
    for item in order['info']['fills']:
        order_book.loc[len(order_book.index)] = [order['datetime'].split('T')[0],
                        order['datetime'].split('T')[1][:-5],
                        order['info']['symbol'],
                        order['info']['side'],
                        item['price'],
                        item['qty'],
                        float(item['price'])*float(item['qty']),
                        order['info']['executedQty'],
                        item['commission']]
    return order_book

In [12]:
def place_order(signal, amount, symbol):
    if signal == 'buy':
        order = exchange.create_market_buy_order(symbol, amount)
    elif signal == 'sell':
        order = exchange.create_market_sell_order(symbol, amount)
    order_book = append_order_book(order)
    order_book = pd.DataFrame(order_book)
    return order_book

In [20]:
place_order(signal = 'buy', amount = '0.3', symbol = 'BTC/USDT')

InvalidNonce: binance {"code":-1021,"msg":"Timestamp for this request was 1000ms ahead of the server's time."}

In [36]:
if 'current_assets' in globals():
    del current_assets

column_names = ['Asset', 'Free Value', 'Locked Value', 'Value']

current_assets = pd.DataFrame(columns=column_names)
current_assets

,Asset,Free Value,Locked Value,Value


In [37]:
import numpy as np

def get_current_price(symbol):
    try:
        ticker = exchange.fetch_ticker(symbol)
        
        current_price = ticker.get('last', np.nan)
        return current_price
    except:
        return np.nan

In [421]:
import numpy as np

for item in balance['info']['balances'][:10]:
    if item['asset'] == 'USDT':
        value = item['free']
    else:
        symbol = f"{item['asset']}/USDT"
        current_price = get_current_price(symbol)
        value = float(item['free'])*float(current_price)

    current_assets.loc[len(current_assets.index)] = [item['asset'], item['free'], item['locked'], value]
    
current_assets

,Asset,Free Value,Locked Value,Value
0,ETH,0.64120000,0.00000000,NaN
1,BTC,1.12300000,0.00000000,NaN
2,LTC,8.00000000,0.00000000,NaN
3,BNB,1.00000000,0.00000000,NaN
4,USDT,3630.00161660,0.00000000,3630.00161660
5,TRX,4026.00000000,0.00000000,NaN
6,XRP,973.00000000,0.00000000,NaN
7,NEO,55.00000000,0.00000000,NaN
8,QTUM,239.00000000,0.00000000,NaN
9,EOS,1095.00000000,0.00000000,NaN
